In [25]:
import pandas as pd

In [26]:
id1 = pd.read_csv('data/wikidata_paintings_ids.csv')
id2 = pd.read_csv('data/wikidata_paintings_ids_final.csv')

In [27]:
id1 = id1.drop_duplicates(subset='item')
id1= id1.drop(columns=['title', 'author_name'])

In [28]:
id_concat = pd.concat([id1, id2], ignore_index=True)
id_concat = id_concat.sort_values(by='location_wikidata')
id_concat = id_concat.drop_duplicates(subset='item', keep='first')

In [29]:

display(id_concat[id_concat['location_wikidata'].isna()])

,item,author_wikidata,location_wikidata
0,http://www.wikidata.org/entity/Q607761,http://www.wikidata.org/entity/Q5432,NaN
38,http://www.wikidata.org/entity/Q276174,http://www.wikidata.org/entity/Q240526,NaN
71,http://www.wikidata.org/entity/Q245231,http://www.wikidata.org/entity/Q637420,NaN
91,http://www.wikidata.org/entity/Q777080,http://www.wikidata.org/entity/Q159606,NaN
96,http://www.wikidata.org/entity/Q287169,http://www.wikidata.org/entity/Q159606,NaN
...,...,...,...
171811,http://www.wikidata.org/entity/Q100370086,http://www.wikidata.org/entity/Q3348980,NaN
171812,http://www.wikidata.org/entity/Q100370087,http://www.wikidata.org/entity/Q3348980,NaN
171813,http://www.wikidata.org/entity/Q100370088,http://www.wikidata.org/entity/Q3348980,NaN
171814,http://www.wikidata.org/entity/Q100370089,http://www.wikidata.org/entity/Q3348980,NaN


In [30]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# Sample large batch data
data = {
    'item': [
        'http://www.wikidata.org/entity/Q607761',
        'http://www.wikidata.org/entity/Q276174',
        'http://www.wikidata.org/entity/Q245231',
        'http://www.wikidata.org/entity/Q777080',
        'http://www.wikidata.org/entity/Q287169',
        # Add more items as needed...
    ],
    'author_wikidata': [
        'http://www.wikidata.org/entity/Q5432',
        'http://www.wikidata.org/entity/Q240526',
        'http://www.wikidata.org/entity/Q637420',
        'http://www.wikidata.org/entity/Q159606',
        'http://www.wikidata.org/entity/Q159606',
        # Add more authors as needed...
    ],
    'location_wikidata': [None] * 5  # Initialize with None
}

# Create DataFrame
df = pd.DataFrame(data)

# Function to create and execute a SPARQL query for P276 locations
def fetch_locations_via_sparql(item_ids):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    # Format the SPARQL query
    item_values = " ".join(f"wd:{item_id}" for item_id in item_ids)
    query = f"""
    SELECT ?item ?location WHERE {{
      VALUES ?item {{ {item_values} }}
      ?item wdt:P276 ?location .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Execute the query and parse results
    try:
        results = sparql.query().convert()
        locations = {result['item']['value']: result['location']['value'] for result in results["results"]["bindings"]}
        return locations
    except Exception as e:
        print(f"Error with SPARQL query: {e}")
        return {}

# Convert item URLs to Wikidata IDs for querying
df['item_id'] = df['item'].apply(lambda url: url.split('/')[-1])

# Fetch locations in batches (adjust batch size as needed)
batch_size = 50
for i in range(0, len(df), batch_size):
    batch_ids = df['item_id'].iloc[i:i + batch_size].tolist()
    locations = fetch_locations_via_sparql(batch_ids)
    
    # Map the fetched locations back to the DataFrame
    df.loc[df['item_id'].isin(batch_ids), 'location_wikidata'] = df['item'].map(locations)

# Drop the temporary 'item_id' column if no longer needed
df.drop(columns=['item_id'], inplace=True)


In [31]:

# Display or save the result
display(df)
# Optionally, save to a CSV or other format
# df.to_csv("updated_wikidata_locations.csv", index=False)

,item,author_wikidata,location_wikidata
0,http://www.wikidata.org/entity/Q607761,http://www.wikidata.org/entity/Q5432,NaN
1,http://www.wikidata.org/entity/Q276174,http://www.wikidata.org/entity/Q240526,NaN
2,http://www.wikidata.org/entity/Q245231,http://www.wikidata.org/entity/Q637420,NaN
3,http://www.wikidata.org/entity/Q777080,http://www.wikidata.org/entity/Q159606,NaN
4,http://www.wikidata.org/entity/Q287169,http://www.wikidata.org/entity/Q159606,NaN
